In [1]:
pip install pytesseract

In [2]:
pip install fuzzywuzzy python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.8 MB/s eta 0:00:00


In [3]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [4]:
# Data Handling
import pandas as pd
import numpy as np

# Optical Character Recognition (OCR)
import pytesseract
from PIL import Image

# Fuzzy Matching for Address Correction
from fuzzywuzzy import process

# Machine Learning Model (Logistic Regression)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Web Framework (Optional - for UI)
import streamlit as st
import flask

# Visualization (Optional - for analysis)
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings (Ignore Deprecation Warnings)
import warnings
warnings.filterwarnings("ignore")

print("Step 1: All necessary libraries imported successfully! 🚀")


Step 1: All necessary libraries imported successfully! 🚀


In [5]:
# Load the dataset
file_path = "pincode.csv"  # Update with correct path
df = pd.read_csv(file_path)

# Display basic info
print("✅ Dataset Loaded Successfully!\n")
print(df.info())

# Display first few rows
print("\n🔹 Preview of the dataset:")
print(df.head())

# Handling missing values (drop rows with empty values)
df.dropna(inplace=True)

# Standardizing text columns (uppercase + remove extra spaces)
text_columns = ["officename", "district", "statename"]
for col in text_columns:
    df[col] = df[col].astype(str).str.upper().str.strip()

# Save cleaned dataset
df.to_csv("pincode.csv", index=False)

print("\n✅ Data Preprocessing Completed! Ready for the next step.")


✅ Dataset Loaded Successfully!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165300 entries, 0 to 165299
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   officename  165300 non-null  object
 1   pincode     165300 non-null  int64 
 2   district    164938 non-null  object
 3   statename   164938 non-null  object
dtypes: int64(1), object(3)
memory usage: 5.0+ MB
None

🔹 Preview of the dataset:
    officename  pincode district  statename
0      Telo BO   828403   BOKARO  JHARKHAND
1    Karmatanr   828404   BOKARO  JHARKHAND
2  Bharkhar BO   825106  GIRIDIH  JHARKHAND
3     Chino BO   825106  GIRIDIH  JHARKHAND
4   Jamtara BO   825106  GIRIDIH  JHARKHAND

✅ Data Preprocessing Completed! Ready for the next step.


In [6]:
!apt-get install -y tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [7]:
import pytesseract
from PIL import Image

# Function to extract text from an image
def extract_text_from_image(image_path):
    try:
        image = Image.open(image_path)  # Load image
        extracted_text = pytesseract.image_to_string(image)  # Apply OCR
        return extracted_text
    except Exception as e:
        print(f"Error: {e}")
        return None

# Test OCR with a sample image
image_path = "image.png"  # Update with actual image path
extracted_text = extract_text_from_image(image_path)

print("\n🔹 Extracted Address Text:\n")
print(extracted_text)


Error: [Errno 2] No such file or directory: 'image.png'

🔹 Extracted Address Text:

None


In [8]:
import re
import pandas as pd
from fuzzywuzzy import process


df = pd.read_csv("pincode.csv")
df['pincode'] = df['pincode'].astype(str)


def validate_address(officename, pincode, district, statename):
    if not officename or not pincode or not district or not statename:
        return {"error": "Invalid input: missing address details ❌"}


    matched_entry = df[
        (df['officename'].str.lower() == officename.lower()) &
        (df['pincode'] == str(pincode)) &
        (df['district'].str.lower() == district.lower()) &
        (df['statename'].str.lower() == statename.lower())
    ]

    if not matched_entry.empty:
        return {
            "officename": officename,
            "pincode": pincode,
            "district": district,
            "statename": statename,
            "status": "Valid Address ✅"
        }


    best_officename_match = process.extractOne(officename, df['officename'])

    if best_officename_match:
        corrected_officename = best_officename_match[0]


        corrected_entry = df[df['officename'].str.lower() == corrected_officename.lower()].iloc[0]

        return {
            "officename": corrected_entry['officename'],
            "pincode": corrected_entry['pincode'],
            "district": corrected_entry['district'],
            "statename": corrected_entry['statename'],
            "status": "Corrected Address 🔄"
        }

    return {"error": "No suitable correction found ❌"}


def extract_address_details(ocr_text):
    print("🔹 Raw Extracted OCR Text:\n", ocr_text)


    lines = [line.strip() for line in ocr_text.strip().split("\n") if line.strip()]
    print("\n🔹 Extracted Lines:\n", lines)

    address_data = {"From": {}, "To": {}}
    current_section = None
    temp_address = []

    for line in lines:
        if "From:" in line:
            current_section = "From"
            temp_address = []
            continue
        elif "To:" in line:

            if temp_address and current_section:
                address_data[current_section] = process_address(temp_address)
            current_section = "To"
            temp_address = []
            continue
        elif "POSTCARD" in line:
            break

        if current_section:
            temp_address.append(line)


    if temp_address and current_section:
        address_data[current_section] = process_address(temp_address)

    print("\n🔹 Structured Address Data:\n", address_data)
    return address_data

def process_address(address_lines):
    """Processes multi-line address data into structured format."""
    officename, district, pincode, statename = None, None, None, None

    if len(address_lines) >= 3:
        officename = address_lines[1].strip().upper()
        last_line = address_lines[-1].strip().upper()
        statename = last_line if last_line.isalpha() else None

        district_pincode = address_lines[-2].strip()
        pincode_match = re.search(r"\b\d{6}\b", district_pincode)

        if pincode_match:
            pincode = pincode_match.group(0)
            district = district_pincode.replace(pincode, "").strip().upper()

    return {
        "officename": officename,
        "district": district,
        "pincode": pincode,
        "statename": statename,
    }

# Example OCR Input
extracted_text = """
From:
Aishwarya Patwatkar,
samta nagar,
Thane 421306,
Maharashtra

To:
Chirag Sawant,
Sasne B.O,
Thane 421401,
Maharashtra

POSTCARD
"""

# Step 1: Extract Address Data
structured_address = extract_address_details(extracted_text)

# Step 2: Validate & Correct Address Data
validated_addresses = {
    "From": validate_address(**structured_address["From"]),
    "To": validate_address(**structured_address["To"]),
}

# Output results
print("\n✅ **Final Validated & Corrected Address Data:**")
print(validated_addresses)


🔹 Raw Extracted OCR Text:
 
From:
Aishwarya Patwatkar,
samta nagar,
Thane 421306,
Maharashtra

To:
Chirag Sawant,
Sasne B.O,
Thane 421401,
Maharashtra

POSTCARD


🔹 Extracted Lines:
 ['From:', 'Aishwarya Patwatkar,', 'samta nagar,', 'Thane 421306,', 'Maharashtra', 'To:', 'Chirag Sawant,', 'Sasne B.O,', 'Thane 421401,', 'Maharashtra', 'POSTCARD']

🔹 Structured Address Data:
 {'From': {'officename': 'SAMTA NAGAR,', 'district': 'THANE ,', 'pincode': '421306', 'statename': 'MAHARASHTRA'}, 'To': {'officename': 'SASNE B.O,', 'district': 'THANE ,', 'pincode': '421401', 'statename': 'MAHARASHTRA'}}

✅ **Final Validated & Corrected Address Data:**
{'From': {'officename': 'NAGAR', 'pincode': '752109', 'district': 'PURI', 'statename': 'ODISHA', 'status': 'Corrected Address 🔄'}, 'To': {'officename': 'SASNE B.O', 'pincode': '421401', 'district': 'THANE', 'statename': 'MAHARASHTRA', 'status': 'Corrected Address 🔄'}}


In [9]:
import re
import pandas as pd
import pytesseract
from PIL import Image
from fuzzywuzzy import process

# Load dataset (make sure pincode.csv is in the same folder)
file_path = "pincode.csv"  # No leading slash
df = pd.read_csv(file_path)
df['pincode'] = df['pincode'].astype(str)

def extract_text_from_image(image_path):
    """Extract text from the given image using OCR."""
    try:
        image = Image.open(image_path)
        ocr_text = pytesseract.image_to_string(image)
        print("🔹 Extracted OCR Text:\n", ocr_text)
        return ocr_text
    except Exception as e:
        print(f"❌ Error extracting text from image: {e}")
        return ""

def extract_address_details(ocr_text):
    """Parses extracted OCR text into structured address format."""
    print("\n🔹 Raw Extracted OCR Text:\n", ocr_text)

    lines = [line.strip() for line in ocr_text.strip().split("\n") if line.strip()]
    print("\n🔹 Extracted Lines:\n", lines)

    address_data = {"From": {}, "To": {}}
    current_section = None
    temp_address = []

    for line in lines:
        if "From:" in line:
            current_section = "From"
            temp_address = []
            continue
        elif "To:" in line:
            if temp_address and current_section:
                address_data[current_section] = process_address(temp_address)
            current_section = "To"
            temp_address = []
            continue
        elif "POSTCARD" in line:
            break

        if current_section:
            temp_address.append(line)

    if temp_address and current_section:
        address_data[current_section] = process_address(temp_address)

    print("\n🔹 Structured Address Data:\n", address_data)
    return address_data

def process_address(address_lines):
    """Processes multi-line address data into structured format."""
    officename, district, pincode, statename = None, None, None, None

    if len(address_lines) >= 3:
        officename = address_lines[1].strip().upper()
        last_line = address_lines[-1].strip().upper()
        statename = last_line if last_line.isalpha() else None

        district_pincode = address_lines[-2].strip()
        pincode_match = re.search(r"\b\d{6}\b", district_pincode)

        if pincode_match:
            pincode = pincode_match.group(0)
            district = district_pincode.replace(pincode, "").strip().upper()

    return {
        "officename": officename,
        "district": district,
        "pincode": pincode,
        "statename": statename,
    }

def validate_address(officename, pincode, district, statename):
    """Validates and corrects address details using fuzzy matching."""
    if not officename or not pincode or not district or not statename:
        return {"error": "Invalid input: missing address details ❌"}

    matched_entry = df[
        (df['officename'].str.lower() == officename.lower()) &
        (df['pincode'] == str(pincode)) &
        (df['district'].str.lower() == district.lower()) &
        (df['statename'].str.lower() == statename.lower())
    ]

    if not matched_entry.empty:
        return {
            "officename": officename,
            "pincode": pincode,
            "district": district,
            "statename": statename,
            "status": "Valid Address ✅"
        }

    best_officename_match = process.extractOne(officename, df['officename'])

    if best_officename_match:
        corrected_officename = best_officename_match[0]
        corrected_entry = df[df['officename'].str.lower() == corrected_officename.lower()].iloc[0]

        return {
            "officename": corrected_entry['officename'],
            "pincode": corrected_entry['pincode'],
            "district": corrected_entry['district'],
            "statename": corrected_entry['statename'],
            "status": "Corrected Address 🔄"
        }

    return {"error": "No suitable correction found ❌"}

def has_all_fields(address):
    return (
        isinstance(address, dict) and
        all(key in address and address[key] for key in ["officename", "pincode", "district", "statename"])
    )

# --- MAIN EXECUTION ---

image_path = "image.png"  # Make sure this image exists in your directory

ocr_extracted_text = extract_text_from_image(image_path)

structured_address = extract_address_details(ocr_extracted_text)

# Debug prints
print("\n📌 From Address:", structured_address.get("From"))
print("📌 To Address:", structured_address.get("To"))

validated_addresses = {
    "From": validate_address(**structured_address["From"]) if has_all_fields(structured_address.get("From")) else {"error": "Invalid 'From' address ❌"},
    "To": validate_address(**structured_address["To"]) if has_all_fields(structured_address.get("To")) else {"error": "Invalid 'To' address ❌"},
}

print("\n✅ **Final Validated & Corrected Address Data:**")
print(validated_addresses)


❌ Error extracting text from image: [Errno 2] No such file or directory: 'image.png'

🔹 Raw Extracted OCR Text:
 

🔹 Extracted Lines:
 []

🔹 Structured Address Data:
 {'From': {}, 'To': {}}

📌 From Address: {}
📌 To Address: {}

✅ **Final Validated & Corrected Address Data:**
{'From': {'error': "Invalid 'From' address ❌"}, 'To': {'error': "Invalid 'To' address ❌"}}


In [10]:
import re
import pandas as pd
import pytesseract
from PIL import Image
from fuzzywuzzy import process

# Load the dataset
file_path = "pincode.csv"  # Make sure this file exists in your working directory
df = pd.read_csv(file_path)
df['pincode'] = df['pincode'].astype(str)

def extract_text_from_image(image_path):
    """Extract text from the given image using OCR."""
    try:
        image = Image.open(image_path)
        ocr_text = pytesseract.image_to_string(image)
        print("🔹 Extracted OCR Text:\n", ocr_text)
        return ocr_text
    except Exception as e:
        print(f"❌ Error extracting text from image: {e}")
        return ""

def extract_address_details(ocr_text):
    """Parses extracted OCR text into structured address format."""
    print("\n🔹 Raw Extracted OCR Text:\n", ocr_text)
    lines = [line.strip() for line in ocr_text.strip().split("\n") if line.strip()]
    print("\n🔹 Extracted Lines:\n", lines)

    address_data = {"From": {}, "To": {}}
    current_section = None
    temp_address = []

    for line in lines:
        if "From:" in line:
            current_section = "From"
            temp_address = []
            continue
        elif "To:" in line:
            if temp_address and current_section:
                address_data[current_section] = process_address(temp_address)
            current_section = "To"
            temp_address = []
            continue
        elif "POSTCARD" in line:
            break

        if current_section:
            temp_address.append(line)

    if temp_address and current_section:
        address_data[current_section] = process_address(temp_address)

    print("\n🔹 Structured Address Data:\n", address_data)
    return address_data

def process_address(address_lines):
    """Processes multi-line address data into structured format."""
    officename, district, pincode, statename = None, None, None, None

    if len(address_lines) >= 3:
        officename = address_lines[1].strip().upper()
        last_line = address_lines[-1].strip().upper()
        statename = last_line if last_line.isalpha() else None

        district_pincode = address_lines[-2].strip()
        pincode_match = re.search(r"\b\d{6}\b", district_pincode)

        if pincode_match:
            pincode = pincode_match.group(0)
            district = district_pincode.replace(pincode, "").strip().upper()

    return {
        "officename": officename,
        "district": district,
        "pincode": pincode,
        "statename": statename,
    }

def validate_address(officename, pincode, district, statename):
    """Validates and corrects address details using fuzzy matching."""
    if not officename or not pincode or not district or not statename:
        return {"error": "Invalid input: missing address details ❌"}

    matched_entry = df[
        (df['officename'].str.lower() == officename.lower()) &
        (df['pincode'] == str(pincode)) &
        (df['district'].str.lower() == district.lower()) &
        (df['statename'].str.lower() == statename.lower())
    ]

    if not matched_entry.empty:
        return {
            "officename": officename,
            "pincode": pincode,
            "district": district,
            "statename": statename,
            "status": "Valid Address ✅"
        }

    best_officename_match = process.extractOne(officename, df['officename'])

    if best_officename_match:
        corrected_officename = best_officename_match[0]
        corrected_entry = df[df['officename'].str.lower() == corrected_officename.lower()].iloc[0]

        return {
            "officename": corrected_entry['officename'],
            "pincode": corrected_entry['pincode'],
            "district": corrected_entry['district'],
            "statename": corrected_entry['statename'],
            "status": "Corrected Address 🔄"
        }

    return {"error": "No suitable correction found ❌"}

def has_all_fields(address):
    return (
        isinstance(address, dict) and
        all(key in address and address[key] for key in ["officename", "pincode", "district", "statename"])
    )

# --- MAIN EXECUTION ---

image_path = "image2.png"  # Make sure this file exists in your directory

ocr_extracted_text = extract_text_from_image(image_path)

structured_address = extract_address_details(ocr_extracted_text)

# Debug prints
print("\n📌 From Address:", structured_address.get("From"))
print("📌 To Address:", structured_address.get("To"))

# Safe validation with field checks
validated_addresses = {
    "From": validate_address(**structured_address["From"]) if has_all_fields(structured_address.get("From")) else {"error": "Invalid or Incomplete 'From' Address ❌"},
    "To": validate_address(**structured_address["To"]) if has_all_fields(structured_address.get("To")) else {"error": "Invalid or Incomplete 'To' Address ❌"},
}

# Final Output
print("\n✅ **Final Validated & Corrected Address Data:**")
print(validated_addresses)


❌ Error extracting text from image: [Errno 2] No such file or directory: 'image2.png'

🔹 Raw Extracted OCR Text:
 

🔹 Extracted Lines:
 []

🔹 Structured Address Data:
 {'From': {}, 'To': {}}

📌 From Address: {}
📌 To Address: {}

✅ **Final Validated & Corrected Address Data:**
{'From': {'error': "Invalid or Incomplete 'From' Address ❌"}, 'To': {'error': "Invalid or Incomplete 'To' Address ❌"}}
